## Used to simulate the entire flow (Hybrid HHL and Time Locked Puzzle)

```markdown
![Workflow Diagram](./diagram_of_scheme.png)
```

In [ ]:
from enhanced_hybrid_hhl import (
    QuantumLinearSystemProblem,
    run_braket_projection_submission,
    run_braket_projection_retrieval
)